<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/4)_Consigna_35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma?***

Realizo la instalación de todo lo que utilizare en el desarrollo de la consulta.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!pip install pyarrow

Importo las librerias.

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 

In [4]:
import pyarrow

Creo el SparkSession.

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo a mi google drive para acceder al archivo.

In [6]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Lo pase a .parquet desde el local al archivo, porque desde colab no me dejaba, me saltaba un error con el drive.


Leo el archivo.

In [7]:
logs = sqlContext.read.parquet('gdrive/My Drive/TP2 - Datos/logs.parquet')

In [8]:
rddLogs = logs.rdd

Limpio el rdd.

In [9]:
rddLogs = rddLogs.filter(lambda x: x.contributor_username != None and x.title != None)

Filtro por bloqueados.

In [10]:
rddBloq = rddLogs.filter(lambda x: x.logtype == 'block')

Averiguo cuales fueron los usuarios bloqueados y en que fecha. Me quedo con ese formato de RDD.

In [11]:
rddBlock = rddBloq.map(lambda x: (x.title.split(':')[1],pd.to_datetime(x.timestamp)))

Realizo un take para ver que cual es el formato.

In [12]:
rddBlock.take(2)

[('212.55.30.157', Timestamp('2004-12-23 08:28:32+0000', tz='UTC')),
 ('201.241.228.15', Timestamp('2004-12-27 17:54:50+0000', tz='UTC'))]

Me quedo con la fecha del primer bloqueo del usuario.

In [13]:
rddBlock = rddBlock.reduceByKey(lambda a,b: a if a < b else b)

Filtro por registro de usuarios (durante todo el desarrollo trabajo con los usuarios ***registrados***).

In [14]:
rddNewUser = rddLogs.filter(lambda x: x.logtype == 'newusers')

Me quedo con un RDD que tiene el username del usuario creado y la fecha en que se creo el usuario.

In [15]:
rddUser =  rddNewUser.map(lambda x: (x.contributor_username,pd.to_datetime(x.timestamp)))

Realizo un take para ver que cual es el formato.

In [16]:
rddUser.take(2)

[('Yliana', Timestamp('2005-09-07 22:33:56+0000', tz='UTC')),
 ('BrokenSegue', Timestamp('2005-09-07 22:50:58+0000', tz='UTC'))]

Me quedo con la primer fecha de registro del usuario.

In [17]:
rddUser = rddUser.reduceByKey(lambda a,b: a if a < b else b)

Realizo un take para ver que cual es el formato.

In [18]:
rddUser.take(2)

[('Heavymetal~eswiki', Timestamp('2005-09-08 23:56:22+0000', tz='UTC')),
 ('MENDIOLA´S', Timestamp('2005-09-09 00:09:06+0000', tz='UTC'))]

Realizo un join para tener (username, (fecha de alta, fecha de bloqueo)).

In [19]:
rddJoin = rddUser.join(rddBlock)

Realizo un take para ver que cual es el formato.

In [20]:
rddJoin.take(2)

[('Phillip ray xd',
  (Timestamp('2020-07-12 22:37:06+0000', tz='UTC'),
   Timestamp('2020-07-12 22:38:51+0000', tz='UTC'))),
 ('ホセ・マヌエルバスケスがアリアナグランデを脅かす',
  (Timestamp('2021-07-04 09:26:35+0000', tz='UTC'),
   Timestamp('2021-07-04 09:26:58+0000', tz='UTC')))]

Importo numpy para utilizar operaciones con el timedelta.

In [21]:
import numpy as np

Calculo las duraciones de los usuarios haciendo fecha de bloqueo - fecha de alta en el sitio. Filtro con >0 a cero, dado que hay bugs en los datos. *Realice esta consulta dado que me daban resultados extraños, ya que habia numeros muy chicos y a su vez muy grandes, y en teoria todos deberian ser mayores a cero, dado que al trabajar con usuarios registrados, antes de ser bloqueados tendrian que ser registrados.*

In [22]:
rddDelta = rddJoin.map(lambda x:(x[1][1]-x[1][0])).filter(lambda x: x>np.timedelta64(0))

**Resultado**

In [23]:
rddDelta.min()

Timedelta('0 days 00:00:04')

En el desarrollo de la consulta, se muestran todos los pasos intermedios, pero puede ser llevada acabo en menos lineas de codigo.